这篇笔记是对计算机视觉领域中的图片分类问题，及其以数据驱动的实现的简要介绍。文章的受众为对这些问题不熟悉的读者。下面是内容提纲：
* 图像分类、数据驱动实现、作业管线的简介；
* 最近邻算法；
  * k最近邻算法；
* 验证集、交叉验证、超参数调参；
* 最近邻算法的优缺点；
* 总结；
* 总结：k最近邻算法的应用；
* 深入阅读材料。

## 图像分类

**动机：**我们将在本节中介绍图像识别问题——即用一个给定的分类集合对输入图片做分类标记。这个问题尽管看上去并不复杂，但它是计算机视觉领域的核心问题之一，也大量应用于日常业务中。此外，随着课程的深入，我们会发现，很多看起来不同的问题（如物体检测、分割）都可以简化为图像分量问题。

**举例：**在下面的图像分类模型中，我们使用一张图片作为输入，以$\{\textrm{cat, dog, hat, mug}\}$作为分类集合，并计算出图片属于集合中这四个元素的概率。虽然我们看到的是一张猫的图片，但是对于计算机来说，图片只是一个很大的三维数组。以这种图片为例，它的宽为$248$像素，高为$400$像素，每个像素有Red, Green, Blue三个通道（即RGB颜色）。因此，这张图片由$248\times400\times3$——约$297600$个数字组成，每个数字能够从$0$（黑色）取到$255$（白色）。我们的任务，就是将这近三十万个数字总结为一个简单的标记——比如$\textrm{cat}$。

----
<img src="assets/m01_img01.png" alt="" style="width: 400;"/>

图像分类的目标就是通过输入图片得到一个分类标记（或得到一个在分类集合上的概率分布，如上图所示）。图片在计算机中表示为一个三维数组，数组的每个元素在$[0,255]$取值，数组的大小为$\textrm{width}\times\textrm{height}\times3$，其中的$3$表示有三个颜色通道（Red, Green, Blue）。

----

**挑战：**尽管对人类来说，通过视觉识别一个诸如“猫咪”的概念是很简单的，但对于计算机视觉算法来说却是不小的挑战。下面给出一个难点列表，我们要在脑中形成一个概念——对于计算机来说，“图像”只是一个由颜色通道的明暗描述的三维数组而已：
* **视角变化：**从不同的角度取景同一个物体，将会得到各种不同的关于这个物体的图像；
* **尺寸缩放：**物体展示出来的视觉大小通常会改变（这不仅体现在同一个物体的图像上，真实世界中某一类物体本身也会有不同的大小）；
* **形状改变：**真实世界中物体的形状并不是永远不变的，它们的形状通常能够进行很大改变；
* **遮挡：**目标物体可能被其他物体遮挡，有时仅露出一小部分（比如通过较少的像素进行辨认）；
* **光照条件：**光照对像素取值的影响非常剧烈；
* **背景干扰：**目标物体可能融入到环境背景中，这将会增加辨认难度；
* **类内差异：**目标类型通常会包含很多互不相同的个体，比如*椅子*，在日常生活中我们会见到各式各样的椅子，它们所呈现的图像也是各不相同的。

